In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("kssanjaynithish03/retinal-fundus-images")

print("Path to dataset files", path)

100%|██████████| 2.32G/2.32G [01:05<00:00, 37.7MB/s]

Extracting files...


Path to dataset files: /Users/aadi/.cache/kagglehub/datasets/kssanjaynithish03/retinal-fundus-images/versions/3


In [1]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms, models
import torch.optim as optim
import os

KeyboardInterrupt: 

In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5])
    ])
}

In [ ]:
data_dir = r'C:\Users\krish\.cache\kagglehub\datasets\kssanjaynithish03\retinal-fundus-images\versions\3\Retinal Fundus Images'

image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=4, shuffle=True, num_workers=4) for x in ['train', 'val']}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
print(dataset_sizes)

class_names = image_datasets['val'].classes
class_names

{'train': 20077, 'val': 433}


['1.Dry AMD',
 '10.Glaucoma',
 '11.Normal Fundus',
 '2.Wet AMD',
 '3.Mild DR',
 '4.Moderate DR',
 '5.Severe DR',
 '6.Proliferate DR',
 '7.Cataract',
 '8.Hypertensive Retinopathy',
 '9.Pathological Myopia']

In [ ]:
base_model = models.efficientnet_b4(pretrained=True)

base_model.classifier = nn.Identity()

c:\Users\krish\Downloads\Aadi Projectcs\retinal-fundus-disorder-detection\.venv\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\krish\Downloads\Aadi Projectcs\retinal-fundus-disorder-detection\.venv\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B4_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B4_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
class FundusClassifier(nn.Module):
    def __init__(self, base_model, num_classes):
        super(FundusClassifier, self).__init__()
        self.base_model = base_model
        self.batch_norm = nn.LayerNorm(1792, eps=1e-3 )#momentum=0.99)
        self.fc1 = nn.Linear(1792, 256)
        self.dropout = nn.Dropout(p=0.45)
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        #print(f"Input shape before base model: {x.shape}")
        x = self.base_model(x)
        #if x.shape[0] > 1:
            #print(f"Output shape after base model: {x.shape}")  # Debugging output
        x = self.batch_norm(x)
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return x
    
num_classes = 11
model = FundusClassifier(base_model, num_classes)

optimizer = optim.Adamax(model.parameters(), lr=0.001)

criterion = nn.CrossEntropyLoss()

print(model)

FundusClassifier(
  (base_model): EfficientNet(
    (features): Sequential(
      (0): Conv2dNormActivation(
        (0): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): SiLU(inplace=True)
      )
      (1): Sequential(
        (0): MBConv(
          (block): Sequential(
            (0): Conv2dNormActivation(
              (0): Conv2d(48, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=48, bias=False)
              (1): BatchNorm2d(48, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
              (2): SiLU(inplace=True)
            )
            (1): SqueezeExcitation(
              (avgpool): AdaptiveAvgPool2d(output_size=1)
              (fc1): Conv2d(48, 12, kernel_size=(1, 1), stride=(1, 1))
              (fc2): Conv2d(12, 48, kernel_size=(1, 1), stride=(1, 1))
              (activation): SiLU(inplace=True)
            

In [ ]:
has_mps = torch.backends.mps.is_built()
has_cuda = torch.cuda.is_available()

device = "mps" if has_mps else "cuda" if has_cuda else "cpu"
print(f"Using device: {device}")
model = model.to(device)
inputs, labels = next(iter(dataloaders['train']))

print(f"Batch size: {inputs.shape}")  # Expected: [4, 3, 224, 224] for RGB images
print(f"Labels: {labels}")
num_epochs = 10

for epoch in range(num_epochs):
    for phase in ['train', 'val']:
        if phase == 'train':
            model.train()
        else:
            model.eval()

        running_loss = 0.0
        running_corrects = 0

        for inputs, labels in dataloaders[phase]:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.float() / dataset_sizes[phase]

        print(f'Epoch: {epoch+1} {phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

print("Training Complete!")

Using device: cpu
Batch size: torch.Size([4, 3, 224, 224])
Labels: tensor([ 8,  7, 10,  1])


In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
a = torch.tensor([1.0, 2.0, 3.0]).to(device)
print(a.device)  # Should print "cuda:0" if CUDA is working

ImportError: cannot import name 'amp' from partially initialized module 'torch' (most likely due to a circular import) (c:\Users\krish\Downloads\Aadi Projectcs\retinal-fundus-disorder-detection\.venv\lib\site-packages\torch\__init__.py)

In [ ]:
####   LOOK HERE HARAN   ####
 
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision import models, datasets
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

# Set device

print("CUDA Available:", torch.cuda.is_available())
print("MPS Available:", torch.backends.mps.is_available())  # For Mac M1/M2 users
print("Current Device:", torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu"))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Image size and paths
IMAGE_SIZE = 224
BATCH_SIZE = 32
train_path = r'C:\Users\krish\Downloads\Aadi Projectcs\retinal-fundus-disorder-detection\data\raw\train'
test_path = r'C:\Users\krish\Downloads\Aadi Projectcs\retinal-fundus-disorder-detection\data\raw\test'
val_path = r'C:\Users\krish\Downloads\Aadi Projectcs\retinal-fundus-disorder-detection\data\raw\val'

# Data transformations
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load datasets
train_dataset = datasets.ImageFolder(root=train_path, transform=transform)
val_dataset = datasets.ImageFolder(root=val_path, transform=transform)
test_dataset = datasets.ImageFolder(root=test_path, transform=transform)

# Data loaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

# Load pretrained EfficientNetB4
base_model = models.efficientnet_b4(weights=models.EfficientNet_B4_Weights.IMAGENET1K_V1)
base_model.classifier = nn.Sequential(
    nn.BatchNorm1d(1792),
    nn.Linear(1792, 256),
    nn.ReLU(),
    nn.Dropout(0.45),
    nn.Linear(256, 11)  # Output classes = 11
)
model = base_model.to(device)

# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adamax(model.parameters(), lr=0.001)

# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    train_losses, val_losses = [], []
    train_acc, val_acc = [], []

    for epoch in range(epochs):
        model.train()
        running_loss, correct, total = 0.0, 0, 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item() * images.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

        train_loss = running_loss / len(train_loader.dataset)
        train_acc.append(correct / total)
        train_losses.append(train_loss)

        # Validation
        model.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item() * images.size(0)
                _, preds = torch.max(outputs, 1)
                val_correct += (preds == labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader.dataset)
        val_losses.append(val_loss)
        val_acc.append(val_correct / val_total)

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_acc[-1]:.4f}, Val Loss: {val_loss:.4f}, Val Acc: {val_acc[-1]:.4f}")

    return train_losses, val_losses, train_acc, val_acc

# Train the model
train_losses, val_losses, train_acc, val_acc = train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

cpu


In [ ]:
# Save model
torch.save(model.state_dict(), 'efficientnetb4.pth')

In [ ]:
# Plot Loss & Accuracy
plt.figure()
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Validation Loss')
plt.legend()
plt.title('Loss Over Epochs')
plt.show()

plt.figure()
plt.plot(train_acc, label='Train Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend()
plt.title('Accuracy Over Epochs')
plt.show()

# Evaluate Model
def evaluate_model(model, test_loader):
    model.eval()
    correct, total = 0, 0
    all_preds, all_labels = [], []

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
           
            correct += (preds == labels).sum().item()
            total += labels.size(0)

            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct / total
    print(f"Test Accuracy: {accuracy:.4f}")

    # Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    class_names = train_dataset.classes
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=class_names, yticklabels=class_names, cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()

    # Classification Report
    print("Classification Report:\n", classification_report(all_labels, all_preds, target_names=class_names))

# Run evaluation
evaluate_model(model, test_loader)

git config --global user.email "aaditey.pillai@duke.edu"
git config --global user.name "aaditey932"